Load Libraries

In [144]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix,accuracy_score
from pprint import pprint
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import json
from random import random,randint, shuffle
%matplotlib inline
sess = tf.InteractiveSession()
    

/home/ric/Projects/Python/env/lib/python3.6/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


TensorFlow MLP Class

In [167]:
class TfAnn(object):
    
    def __init__(self):
        self.hidden=[]
        self.np_hidden=[]
        
        self.n_nodes=[]
        self.n_classes = 0
        self.n_hiden_layers = 0

    # create empty network for training
    def init_empty(self,layers,n_classes,size):
        self.n_classes = n_classes
        self.n_hiden_layers = len(layers)
        for i in range(self.n_hiden_layers):
            self.hidden.append({'weights':[],'biases':[]})
            self.np_hidden.append({'weights':[],'biases':[]})
            self.n_nodes.append(layers[i])
        self.output_layer = {'weights':[],'biases':[]}
        self.np_output_layer={"weights":[],"biases":[]}

        for i in range(self.n_hiden_layers):
            self.hidden[i] = {'weights':tf.Variable(tf.random_normal([size, self.n_nodes[i]])),
                      'biases':tf.Variable(tf.random_normal([self.n_nodes[i]]))}
            
        self.output_layer = {'weights':tf.Variable(tf.random_normal([self.n_nodes[-1], self.n_classes])),
                        'biases':tf.Variable(tf.random_normal([self.n_classes]))}

    
    def get_number_of_nodes(self,layer):
        return len(self.hidden[layer]['weights'][0])
    
    # Reinflate network from json description
    def init_json(self,jfile):
        with open(jfile) as json_data:
            tf_data = json.load(json_data)
        self.n_classes = tf_data["n_classes"]
        self.n_hiden_layers = tf_data["n_hiden_layers"]
        self.hidden =tf_data["hidden"]
        
        self.output_layer =tf_data["output"]


        
    def create(self,data,mask):
        # This is the heart of the ann where multiply the data by the wights to the layers 
        for i in range(self.n_hiden_layers):
            project_weights = [list(np.array(a)*b) for a,b in zip(self.hidden[i]['weights'],mask[i])]
            project_biases = [a*b for a,b in zip(self.hidden[i]['biases'],mask[i])]
            layer = tf.add(
                tf.matmul(data,project_weights)
                , project_biases)
            layer= tf.nn.relu(layer)
        output =  tf.add(tf.matmul(layer,self.output_layer['weights']) , self.output_layer['biases'])
        return output
    
    
    #Save a trained network as a json file
    def extract(self,jfile):
        for i in range(self.n_hiden_layers):
            self.np_hidden[i]["weights"] = neural_network_model.hidden[i]["weights"].eval().tolist()
            self.np_hidden[i]["biases"] = neural_network_model.hidden[i]["biases"].eval().tolist()
        self.np_output_layer["weights"] = neural_network_model.output_layer["weights"].eval().tolist()
        self.np_output_layer["biases"] = neural_network_model.output_layer["biases"].eval().tolist()
        with open(jfile,"w") as jout:
            json.dump({"n_classes":self.n_classes, # number of input classifier classes
                       "n_hiden_layers":self.n_hiden_layers, # number of 
                       "hidden":self.np_hidden,# weights and biases
                       # each layer is defined by dict {'weights':[],'biases':[]}
                       "output":self.np_output_layer} # as for hidden
                      ,jout)
        
        

In [168]:
class Gene(object):
    size=0
    mutation_rate=0.0
    
    def __init__(self):
        self.mask=[0 for _ in range(Gene.size)]
        self.fittness=0
        self.accuracy=0
        
    def random(self):
        for i in range(Gene.size):
            self.mask[i] = randint(0, 1)
    
    def cross(self,gene1):
        cross_point = randint(1,Gene.size)
        gene3 = Gene()
        gene4 = Gene()
        gene3.mask=gene1.mask[:cross_point]+self.mask[cross_point:]
        gene4.mask=gene1.mask[cross_point:]+self.mask[:cross_point]
        return gene3, gene4
    
    def mutate(self):
        if random()<Gene.mutation_rate:
            mut_point =  randint(0,Gene.size-1)
            self.mask[mut_point]+=1
            self.mask[mut_point]=self.mask[mut_point]%2
    
    def __lt__(self, other):
        return self.fittness > other.fittness

In [169]:
def test_neural_network(neural_network_model,mask,x_data_test,y_data_test):
    # set up network
    x = tf.placeholder('float')
    prediction = neural_network_model.create(x,mask)    
    
    # ren test data
    y_test_res=(sess.run(prediction,feed_dict={x:x_data_test}))       
    # the correct data
    true_class=np.argmax(y_data_test,1)
    
    # get the index of the outpt array with heighest value
    predicted_class=np.argmax(y_test_res,1)
    
    # calculate confusion matix
    cm = confusion_matrix(true_class,predicted_class)
    cm = cm.astype('float')*10000 / cm.sum(axis=1)[:, np.newaxis]
    cm = np.nan_to_num(cm, copy=True)
    cm = cm.astype('int')
    return accuracy_score(true_class, predicted_class) , cm*0.01

In [170]:
def t_select(size,sub_size):
    pop = list(range(size))
    shuffle(pop)
    x = pop[:sub_size]
    x.sort()
    return x[0]
    

In [171]:
with open("datasets/wine/wine_train.json") as json_data:
    test_dataset = json.load(json_data)

test_x = np.asarray(test_dataset["attribs"])
test_y = np.asarray(test_dataset["target_hot"])

neural_network_model = TfAnn()
neural_network_model.init_json("classifiers/wine-mlp.json")
Gene.size = neural_network_model.get_number_of_nodes(0)
Gene.mutation_rate=0.1
acc,cf = test_neural_network(neural_network_model,[[1 for _ in range(Gene.size)]],test_x,test_y)
print(acc,Gene.size)

1.0 13


In [172]:
gene_pool = []
for _ in range(100):
    a_gene = Gene()
    a_gene.random()
    gene_pool.append(a_gene)


In [173]:
def run_gene(gene):
    gene.accuracy,cf = test_neural_network(neural_network_model,[gene.mask],test_x,test_y)
    gene.fittness = gene.accuracy*(Gene.size-sum(gene.mask))/Gene.size

In [174]:
for gene in gene_pool:
    run_gene(gene)
    

In [175]:
gene_pool.sort()
for g in gene_pool[:20]:
    print(g.fittness,g.accuracy, Gene.size-sum(g.mask))

0.4550379198266522 0.8450704225352113 7
0.4507042253521127 0.7323943661971831 8
0.4452871072589383 0.9647887323943662 6
0.4322860238353196 0.8028169014084507 7
0.4322860238353196 0.8028169014084507 7
0.42903575297941493 0.6971830985915493 8
0.42903575297941493 0.6197183098591549 9
0.42849404117009754 0.795774647887324 7
0.4247020585048754 0.7887323943661971 7
0.4225352112676056 0.9154929577464789 6
0.4062838569880824 0.8802816901408451 6
0.39869989165763814 0.647887323943662 8
0.3943661971830986 0.7323943661971831 7
0.39003250270855905 0.6338028169014085 8
0.3851570964247021 0.5563380281690141 9
0.3829902491874323 0.7112676056338029 7
0.37378114842903576 0.8098591549295775 6
0.37269772481040087 0.6056338028169014 8
0.3705308775731311 0.5352112676056338 9
0.36782231852654385 0.6830985915492958 7


In [176]:
for _ in range( 10):
    new_pool=gene_pool[:20]
    for _ in range(40):
        gene1 = gene_pool[t_select(len(gene_pool),3)]
        gene2 = gene_pool[t_select(len(gene_pool),3)]
        ngene1, ngene2 = gene1.cross(gene2)
        ngene1.mutate()
        ngene2.mutate()
        run_gene(ngene1)
        run_gene(ngene2)
        new_pool.append(ngene1)    
        new_pool.append(ngene2)
    gene_pool = new_pool
    gene_pool.sort()
    g = gene_pool[0]
    print(g.fittness,g.accuracy, Gene.size-sum(g.mask))


0.5633802816901409 0.9154929577464789 8
0.6191765980498375 0.8943661971830986 9
0.6608884073672806 0.8591549295774648 10
0.6608884073672806 0.8591549295774648 10
0.6608884073672806 0.8591549295774648 10
0.6608884073672806 0.8591549295774648 10
0.6933911159263272 0.9014084507042254 10
0.6988082340195017 0.9084507042253521 10
0.6988082340195017 0.9084507042253521 10
0.7329360780065005 0.8661971830985915 11


In [177]:
gene = gene_pool[0]
acc,cf = test_neural_network(neural_network_model,[gene.mask],test_x,test_y)

In [178]:
cf

array([[100.  ,   0.  ,   0.  ],
       [ 14.03,  84.21,   1.75],
       [  0.  ,  26.31,  73.68]])

In [179]:
gene.mask

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1]

In [180]:
 acc

0.8661971830985915

In [181]:
labs=["Alchol", "Malic_Acid", "Ash", "Alcalinity_of_Ash", "Magnesium", "Total_phenols", "Falvanoids", "Nonflavanoid_phenols", "Proanthocyanins", "Color_intensity", "Hue", "OD280", "Proline"]

In [182]:
for l in [i for i,x in enumerate(gene.mask) if x == 1]:
    print(labs[l])

Falvanoids
Proline


if ( Falvanoids <= 0.3632352948188782 ) {
    if ( Color_intensity <= 0.21715017408132553 ) {
        return 2 ( 12 examples )
    }
    else {
        return 2 ( 1 examples )
        return 3 ( 38 examples )
    }
}
else {
    if ( Proline <= 0.3609539121389389 ) {
        return 1 ( 1 examples )
        return 2 ( 40 examples )
    }
    else {
        return 1 ( 46 examples )
        return 2 ( 4 examples )
    }
}